In [ ]:
# Install necessary libraries
!pip install torch torchtext transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.4 MB/s eta 0:00:00


In [ ]:


# Load IMDb dataset
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xzf aclImdb_v1.tar.gz



--2024-12-01 20:25:01--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  3.55MB/s    in 33s     

2024-12-01 20:25:35 (2.40 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



# Part 1: Fine tuning a pre-trained BERT

This code is fine-tuning a pre-trained BERT model on the IMDb dataset for sentiment analysis. It starts with a BERT model that is already pre-trained on a large text corpus (from bert-base-uncased), which has learned general language representations. The code loads this pre-trained model and then trains it further on the IMDb dataset specifically for sentiment analysis (a binary classification task). This process adjusts the weights of the model to better predict positive or negative sentiment based on the IMDb data.

Training BERT from scratch would mean initializing a BERT model with random weights and training it on a very large corpus (like Wikipedia or BooksCorpus) to learn language representations, which is computationally intensive. Fine-tuning, on the other hand, starts from a pre-trained model and requires much less data and computation. So, this code fine-tunes the model rather than training from scratch. All layers of BERT are trainable during fine-tuning, it doesn’t freeze the pre-trained layers. This means that the weights of the entire BERT model (not just the final classification layer) are updated during training on the IMDb dataset.

In [ ]:
# Import libraries and download the data

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Tokenizer and pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.3474839925765991
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.7089992165565491
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.835295557975769
Accuracy: 0.9160


# Part 2: Gradual unfreezing


Gradual unfreezing is a training strategy used in transfer learning. The idea is to fine-tune a pretrained model by initially freezing most of its layers, training the top (or newly added) layers, and progressively unfreezing more layers as training progresses. This allows the model to first learn the new task in a stable manner before updating the weights of the earlier, more general layers.

When you load a pretrained model (like BERT), it has many layers, often with millions of parameters. These pretrained weights are optimized for a general task (e.g., language modeling in BERT). Initially, you freeze all layers except for the task-specific classifier (e.g., the final linear layer) by setting requires_grad=False for those layers. This ensures only the classifier is updated during the initial training phase.


As training progresses, you progressively unfreeze earlier layers in the model, allowing their weights to be fine-tuned. This gradual unfreezing avoids destabilizing the model by ensuring that earlier, more general layers are only updated after the task-specific layers have been refined.

* Each epoch (or after a certain number of epochs), you unfreeze one or more layers.
* This can be done sequentially, layer by layer, or in groups (e.g., unfreezing an entire block of layers).


In [ ]:
# Gradual unfreezing for sentiment analysis and question answering with BERT on IMDb

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Freeze all layers initially
for param in model.parameters():
    param.requires_grad = False

# keep the classifier layer trainable
for name, param in model.named_parameters():
    if "classifier" in name:
        param.requires_grad = True


# Unfreeze layers gradually
def unfreeze_layers(model, epoch):
    # Gradually unfreeze one encoder layer per epoch, starting from the last layer
    if epoch < len(model.bert.encoder.layer):
        for param in model.bert.encoder.layer[-(epoch + 1)].parameters():
            param.requires_grad = True

# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):
        unfreeze_layers(model, epoch)  # Gradual unfreezing

        # Debugging trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        # Define optimizer with trainable parameters
        optimizer = AdamW(trainable_params, lr=2e-5)

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')


# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize the BERT tokenizer and model for sentiment analysis
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Train the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)

# Evaluate the model
evaluate_model(model, test_loader)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.033939775079488754
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.21184031665325165
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.0058774822391569614
Accuracy: 0.9172


# Part 3: Question answering

In [ ]:
import torch
import random
import os
import tarfile
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering
from torch.utils.data import DataLoader, Dataset

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and models
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sentiment_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)
qa_model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad").to(device)

# Prepare data: extract 5 random reviews and labels
def load_reviews(path, label):
    reviews = []
    for filename in os.listdir(path):
        with open(os.path.join(path, filename), "r", encoding="utf-8") as f:
            reviews.append((f.read(), label))
    return reviews

# Function to ask questions about sentiment
def ask_bert_review(text, question):
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt").to(device)
    answer_start_scores, answer_end_scores = qa_model(**inputs, return_dict=False)

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer


# Load positive and negative reviews
positive_reviews = load_reviews("aclImdb/test/pos", label=1)
negative_reviews = load_reviews("aclImdb/test/neg", label=0)


# Combine and sample reviews
all_reviews = positive_reviews + negative_reviews
random.shuffle(all_reviews)
sample_reviews = random.sample(all_reviews, 5)

# Analyze reviews
results = []

for review, label in sample_reviews:
    # Sentiment classification using pretrained bert - not fine tuned
    inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    sentiment_output = sentiment_model(**inputs)
    sentiment = torch.argmax(sentiment_output.logits, dim=1).item()
    sentiment_label = "positive" if sentiment == 1 else "negative"
    actual_label = "positive" if label == 1 else "negative"


    # Store results
    results.append({
        "review": review,
        "predicted_sentiment": sentiment_label,
        "actual_sentiment": actual_label,
        "reason": ask_bert_review(review, f"Why is this review {sentiment_label}?")
    })

# Print results
for i, result in enumerate(results):
    print(f"Review {i + 1}:")
    print(f"Text: {result['review'][:300]}...")  # Truncate for display
    print(f"Predicted Sentiment: {result['predicted_sentiment']}")
    print(f"Actual Sentiment: {result['actual_sentiment']}")
    print(f"Reason: {result['reason']}")
    print("\n")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Review 1:
Text: By 1971 it was becoming more and more obvious that Hammer film studios were on the way out . HANDS OF THE RIPPER is a case in point where even the idea smacks of desperation - The spirit of Jack The Ripper posses his own daughter ! Yeah okay no one was expecting a documentary but this plot seems to ...
Predicted Sentiment: positive
Actual Sentiment: negative
Reason: 


Review 2:
Text: I'm one of those gluttons for punishment when it comes to sitcoms these days-I still will check them out every once in a while.My observation is that most of them aren't very funny even the ones on major networks that are getting high ratings ,I just don't get who is finding them gut busting funny. ...
Predicted Sentiment: positive
Actual Sentiment: negative
Reason: [CLS] why is this review positive ? [SEP]


Review 3:
Text: Ernesto is a man that makes a living out of duping other solid citizens of their hard earned money. Together with Manco, an older man with a lot of experience, he pull

# TO-DO



---

## Parts 1 and 2


* **Experiment with Different Pretrained Models**:
   - Try other transformer-based models like `distilbert-base-uncased` or `roberta-base`. Compare their performance with `bert-base-uncased`.

   roberta-base is by far the worst model to use under these circumstances.  distillbert-based-uncased is by far much quicker.  Bert-based-uncased is the best for accuracy but takes a little bit longer.

* **Adjust Hyperparameters**:
   - Experiment with learning rates (e.g., `1e-5`, `5e-5`, `2e-6`) and batch sizes to observe their impact on model performance.
   - Evaluate how changing the number of epochs affects validation accuracy and loss.

  for distillbert-based-uncased it seems that lower epoch, lower batch sizes seem to help the most.  For bert-based-uncased it seems to  be have a lot of fluctuation to determine what helps and what doesnt but it seems that the higher the epoch the better.  for the robertya base it seems that the higher number of epochs helps and a lower learning rate number helps.

* **Comparative Analysis**:
   - Compare the performance of fine-tuning with gradual unfreezing to fine-tuning with all layers unfrozen from the start.
   - Discuss the computational trade-offs (time vs. accuracy) and performance differences.
  Time vs accruacy is pretty comparable, they both came in around 22 minutes and they both are around a .92 accuracy with bert-based-uncased taking the win with a .9201.  Although with having to test all the different bert-based-uncased sets, it was quite timeconsuming but if you just take that one in particular they are comparable.

---

## Part 3

* **Manual Review**:
   - Manually analyze a few QA outputs to assess whether the model’s reasoning aligns with human judgment. Highlight discrepancies and discuss their causes.


  The model predicts positive when in reality is it negative, possibly from the sarcasm that makes it hard to determine what is going on.

* **Fine-tuned BERT**:
   - Replace the simple pretrained BERT with your best fine-tuned model and/or replace BERT with other transformer-based model for Q&A

   Done

* **Prompt Engineering**:
   - Experiment with different phrasing for the question (e.g., "What makes this review positive/negative?" or "Explain the sentiment of this review."). Discuss how the phrasing affects the model's answers.


   this one was correct with negative to negative, which the last one only predicted positive.  Different ways of stating things can have a positive of negative impact on how it is represented.

* **Entity-Specific Analysis**:
   - Modify the question to focus on specific entities or aspects in the review (e.g., "What does the review say about the acting?" or "Why is the plot criticized?"). Discuss the model’s ability to handle nuanced questions.

  The model failes due to lack of structure or input

In [ ]:
# Import libraries and download the data


import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
import time  # Import the time module
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=1e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} using the A100 GPU")

# distilbert-base-uncased, BATCH_SIZE = 8,lr=1e-5


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 104
Epoch 1/3 Loss: 0.16393446922302246
Number of trainable parameters: 104
Epoch 2/3 Loss: 0.557553768157959
Number of trainable parameters: 104
Epoch 3/3 Loss: 0.08951559662818909
Accuracy: 0.9144
Total time taken: 00:12:21 using the A100 GPU


In [ ]:
# Import libraries and download the data


import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
import time  # Import the time module
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} using the A100 GPU")

# distilbert-base-uncased, BATCH_SIZE = 8,lr=2e-5


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 104
Epoch 1/3 Loss: 0.14459463953971863
Number of trainable parameters: 104
Epoch 2/3 Loss: 0.03000408038496971
Number of trainable parameters: 104
Epoch 3/3 Loss: 0.01974307745695114
Accuracy: 0.9078
Total time taken: 00:12:16 using the A100 GPU


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201


KeyboardInterrupt: 

In [ ]:
# Import libraries and download the data


import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW

from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
import time  # Import the time module
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=5e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using A100 GPU")

# distilbert-base-uncased, BATCH_SIZE = 8,lr=5e-5,EPOCHS = 3

In [ ]:
# Import libraries and download the data


import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW

from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
import time  # Import the time module
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=5e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
print(f"Total time taken: {total_time:.2f} seconds using the L4 GPU")
# distilbert-base-uncased, BATCH_SIZE = 8,lr=5e-5,EPOCHS = 3

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Number of trainable parameters: 104
Epoch 1/3 Loss: 0.4708547592163086
Number of trainable parameters: 104
Epoch 2/3 Loss: 0.551298975944519
Number of trainable parameters: 104
Epoch 3/3 Loss: 0.08882863074541092
Accuracy: 0.8624
Total time taken: 935.42 seconds using the L4 GPU


In [ ]:
# Import libraries and download the data


import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW

from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
import time  # Import the time module
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=5e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")
# distilbert-base-uncased, BATCH_SIZE = 8,lr=5e-5,EPOCHS = 3

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Number of trainable parameters: 104
Epoch 1/3 Loss: 0.32813239097595215
Number of trainable parameters: 104
Epoch 2/3 Loss: 0.08421207219362259
Number of trainable parameters: 104
Epoch 3/3 Loss: 0.1542925238609314
Accuracy: 0.8543
Total time taken: 728.90 seconds using the A100 GPU


In [ ]:
# Import libraries and download the data

# distilbert-base-uncased, BATCH_SIZE = 8,lr=2e-6,EPOCHS = 3

import torch
import time  # Import the time module
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW

from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-6, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 104
Epoch 1/3 Loss: 0.23734691739082336
Number of trainable parameters: 104
Epoch 2/3 Loss: 0.06410857290029526
Number of trainable parameters: 104
Epoch 3/3 Loss: 0.1856422871351242
Accuracy: 0.9046
Total time taken: 00:12:12 Using the A100 GPU


In [ ]:
# Import libraries and download the data

# distilbert-base-uncased, BATCH_SIZE = 16,lr=2e-5,EPOCHS = 3

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
import time  # Import the time module
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 16
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 104
Epoch 1/3 Loss: 0.19335265457630157
Number of trainable parameters: 104
Epoch 2/3 Loss: 0.06016955152153969
Number of trainable parameters: 104
Epoch 3/3 Loss: 0.008294796571135521
Accuracy: 0.9090
Total time taken: 00:10:54 Using the A100 GPU


In [ ]:
# Import libraries and download the data

# distilbert-base-uncased, BATCH_SIZE = 16,lr=2e-5, EPOCHS = 5

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
import time  # Import the time module
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 16
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 104
Epoch 1/5 Loss: 0.3543960452079773
Number of trainable parameters: 104
Epoch 2/5 Loss: 0.03235465660691261
Number of trainable parameters: 104
Epoch 3/5 Loss: 0.031877122819423676
Number of trainable parameters: 104
Epoch 4/5 Loss: 0.0012970701791346073
Number of trainable parameters: 104
Epoch 5/5 Loss: 0.00164070597384125
Accuracy: 0.9130
Total time taken: 00:16:35 Using the A100 GPU


Bert-Based Uncased below

In [ ]:
# Import libraries and download the data

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline
import time  # Import the time module

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Tokenizer and pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=1e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# bert-based-uncased, lr=1e-5, BATCH_SIZE = 8, EPOCHS = 3

In [ ]:
# Import libraries and download the data

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline
import time  # Import the time module

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Tokenizer and pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# bert-based-uncased, lr=2e-5, BATCH_SIZE = 8, EPOCHS = 3

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.6975260972976685
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.5747290849685669
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.37268394231796265
Accuracy: 0.7603
Total time taken: 00:22:09 Using the A100 GPU


In [ ]:
# Import libraries and download the data

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline
import time  # Import the time module

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Tokenizer and pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=1e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# bert-based-uncased, lr=1e-5, BATCH_SIZE = 8, EPOCHS = 3

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.10754819214344025
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.07577671110630035
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.029499314725399017
Accuracy: 0.9201
Total time taken: 00:22:05 Using the A100 GPU


In [ ]:
# Import libraries and download the data

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline
import time  # Import the time module

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Tokenizer and pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=5e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# bert-based-uncased, lr=5e-5, BATCH_SIZE = 8, EPOCHS = 3

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.7242938876152039
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.6792218685150146
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.7002463936805725
Accuracy: 0.5000
Total time taken: 00:22:00 Using the A100 GPU


In [ ]:
# Import libraries and download the data

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline
import time  # Import the time module

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Tokenizer and pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-6, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# bert-based-uncased, lr=2e-6, BATCH_SIZE = 8, EPOCHS = 3

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.6127933859825134
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.19958460330963135
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.026974782347679138
Accuracy: 0.9165
Total time taken: 00:22:04 Using the A100 GPU


In [ ]:
# Import libraries and download the data

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline
import time  # Import the time module

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Tokenizer and pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 16
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-6, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# bert-based-uncased, lr=2e-6, BATCH_SIZE = 16, EPOCHS = 3

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.5284695625305176
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.030269289389252663
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.12884077429771423
Accuracy: 0.9154
Total time taken: 00:20:47 Using the A100 GPU


In [ ]:
# Import libraries and download the data

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline
import time  # Import the time module

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Tokenizer and pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 16
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-6, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# bert-based-uncased, lr=2e-6, BATCH_SIZE = 16, EPOCHS = 5

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Number of trainable parameters: 201
Epoch 1/5 Loss: 0.5571572780609131
Number of trainable parameters: 201
Epoch 2/5 Loss: 0.5179071426391602
Number of trainable parameters: 201
Epoch 3/5 Loss: 0.013046866282820702
Number of trainable parameters: 201
Epoch 4/5 Loss: 0.07614007592201233
Number of trainable parameters: 201
Epoch 5/5 Loss: 0.025075694546103477
Accuracy: 0.9178
Total time taken: 00:33:06 Using the A100 GPU


roberta-base

In [ ]:
# Import libraries and download the data


import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import time  # Import the time module
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for roberta-base
okenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# 'roberta-base, BATCH_SIZE = 8, lr=2e-5, EPOCHS = 3

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.6946337223052979
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.706021249294281
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.7090902328491211
Accuracy: 0.5000
Total time taken: 00:22:02 Using the A100 GPU


In [ ]:
# Import libraries and download the data


import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import time  # Import the time module
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for roberta-base
okenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=1e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# 'roberta-base, BATCH_SIZE = 8, lr=1e-5, EPOCHS = 3

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.6966667175292969
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.7459478378295898
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.4786936044692993
Accuracy: 0.7103
Total time taken: 00:22:11 Using the A100 GPU


In [ ]:
# Import libraries and download the data


import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import time  # Import the time module
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for roberta-base
okenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=5e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# 'roberta-base, BATCH_SIZE = 8, lr=5e-5, EPOCHS = 3

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.7172223329544067
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.6890016198158264
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.686179518699646
Accuracy: 0.5000
Total time taken: 00:22:04 Using the A100 GPU


In [ ]:
# Import libraries and download the data


import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import time  # Import the time module
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for roberta-base
okenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-6, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# 'roberta-base, BATCH_SIZE = 8, lr=2e-6, EPOCHS = 3

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.3895869255065918
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.5510010123252869
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.39616188406944275
Accuracy: 0.7592
Total time taken: 00:22:06 Using the A100 GPU


In [ ]:
# Import libraries and download the data


import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import time  # Import the time module
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for roberta-base
okenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 16
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-6, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# 'roberta-base, BATCH_SIZE = 16, lr=2e-6, EPOCHS = 3

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.5911389589309692
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.42604875564575195
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.45530638098716736
Accuracy: 0.7643
Total time taken: 00:20:48 Using the A100 GPU


In [ ]:
# Import libraries and download the data


import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import time  # Import the time module
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Initialize tokenizer and model for roberta-base
okenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 16
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-6, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# 'roberta-base, BATCH_SIZE = 16, lr=2e-6, EPOCHS = 5

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/5 Loss: 0.5925526022911072
Number of trainable parameters: 201
Epoch 2/5 Loss: 0.5701599717140198
Number of trainable parameters: 201
Epoch 3/5 Loss: 0.6218447089195251
Number of trainable parameters: 201
Epoch 4/5 Loss: 0.37999773025512695
Number of trainable parameters: 201
Epoch 5/5 Loss: 0.2567494213581085
Accuracy: 0.7792
Total time taken: 00:33:13 Using the A100 GPU


In [ ]:
# Best Performance
# Import libraries and download the data

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from transformers import pipeline
import time  # Import the time module

# Start timer
start_time = time.time()

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Training function
def train_model(model, train_loader, optimizer, scheduler, epochs=3):
    model.train()
    for epoch in range(epochs):

        # Listing trainable parameters
        trainable_params = list(filter(lambda p: p.requires_grad, model.parameters()))
        print(f"Number of trainable parameters: {len(trainable_params)}")

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        print(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item()}')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    accuracy = correct_predictions.double() / len(test_loader.dataset)
    print(f'Accuracy: {accuracy:.4f}')


def load_imdb_data(path):
    data = []
    for label in ['pos', 'neg']:
        dir_path = f"{path}/{label}"
        for file in os.listdir(dir_path):
            with open(f"{dir_path}/{file}", 'r', encoding='utf-8') as f:
                text = f.read()
                data.append((1 if label == 'pos' else 0, text))
    return pd.DataFrame(data, columns=['label', 'text'])


# Data Preparation - Tokenizing and padding
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

# Tokenizer and pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)


# Create DataLoaders
BATCH_SIZE = 8
train_dataset = IMDBDataset(train_data, tokenizer)
test_dataset = IMDBDataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Training setup
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=1e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS  # 3 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



# Training the model
train_model(model, train_loader, optimizer, scheduler, EPOCHS)


# Evaluating the model
evaluate_model(model, test_loader)

# End timer
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time

# Convert total time into hours, minutes, seconds
hours = int(total_time // 3600)  # 1 hour = 3600 seconds
minutes = int((total_time % 3600) // 60)  # Get remaining minutes after hours
seconds = int(total_time % 60)  # Get remaining seconds after minutes

# Print the total time in hour:minute:second format
print(f"Total time taken: {hours:02}:{minutes:02}:{seconds:02} Using the A100 GPU")

# bert-based-uncased, lr=1e-5, BATCH_SIZE = 8, EPOCHS = 3

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of trainable parameters: 201
Epoch 1/3 Loss: 0.12851452827453613
Number of trainable parameters: 201
Epoch 2/3 Loss: 0.10446888208389282
Number of trainable parameters: 201
Epoch 3/3 Loss: 0.038200438022613525
Accuracy: 0.9218
Total time taken: 00:22:07 Using the A100 GPU


In [ ]:
import torch
import random
import os
import tarfile
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering
from torch.utils.data import DataLoader, Dataset

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and models
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sentiment_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)
qa_model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad").to(device)

# Prepare data: extract 5 random reviews and labels
def load_reviews(path, label):
    reviews = []
    for filename in os.listdir(path):
        with open(os.path.join(path, filename), "r", encoding="utf-8") as f:
            reviews.append((f.read(), label))
    return reviews

def ask_bert_review(text, question):
    inputs = tokenizer.encode_plus(
        question,
        text,
        add_special_tokens=True,
        return_tensors="pt",
        max_length=512,  # Limit input length
        truncation=True  # Truncate if longer than 512
    ).to(device)

    # Forward pass through QA model
    answer_start_scores, answer_end_scores = qa_model(**inputs, return_dict=False)
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer


# Load positive and negative reviews
positive_reviews = load_reviews("aclImdb/test/pos", label=1)
negative_reviews = load_reviews("aclImdb/test/neg", label=0)


# Combine and sample reviews
all_reviews = positive_reviews + negative_reviews
random.shuffle(all_reviews)
sample_reviews = random.sample(all_reviews, 5)

# Analyze reviews
results = []

for review, label in sample_reviews:
    # Sentiment classification using pretrained bert - not fine tuned
    inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    sentiment_output = sentiment_model(**inputs)
    sentiment = torch.argmax(sentiment_output.logits, dim=1).item()
    sentiment_label = "positive" if sentiment == 1 else "negative"
    actual_label = "positive" if label == 1 else "negative"


    # Store results
    results.append({
        "review": review,
        "predicted_sentiment": sentiment_label,
        "actual_sentiment": actual_label,
        "reason": ask_bert_review(review, f"Why is this review {sentiment_label}?")
    })

# Print results
for i, result in enumerate(results):
    print(f"Review {i + 1}:")
    print(f"Text: {result['review'][:300]}...")  # Truncate for display
    print(f"Predicted Sentiment: {result['predicted_sentiment']}")
    print(f"Actual Sentiment: {result['actual_sentiment']}")
    print(f"Reason: {result['reason']}")
    print("\n")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Review 1:
Text: I hate to comment on something I didn't finish, but if I spare one person what I sat through for almost an hour before turning it off in disgust, it will be worth it. <br /><br />I decided to watch this with an open mind, knowing it was on the bottom 100.<br /><br />Bad idea. I usually love crude hu...
Predicted Sentiment: negative
Actual Sentiment: negative
Reason: it was on the bottom 100


Review 2:
Text: I saw the premier of this movie during the 2005 Phoenix Film Festival and was very impressed with the skill Director, Jeff Hare, exhibited in bringing this timely topic to the screen. The cast of characters meshed perfectly and allowed us to examine the issue of a senior wishing to die on "his own t...
Predicted Sentiment: negative
Actual Sentiment: positive
Reason: if you are not afraid to shed a few tears or laugh at the quips of a family struggling with this issue


Review 3:
Text: I saw this film at the Sundance Film Festival and I too was surprised that it didn

In [ ]:
import torch
import random
import os
from transformers import BertTokenizer, BertForSequenceClassification, BertForQuestionAnswering
from torch.utils.data import DataLoader, Dataset

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and models
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sentiment_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)
qa_model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad").to(device)

# Prepare data: extract reviews and labels
def load_reviews(path, label):
    reviews = []
    for filename in os.listdir(path):
        with open(os.path.join(path, filename), "r", encoding="utf-8") as f:
            reviews.append((f.read(), label))
    return reviews

# QA function for entity-specific questions
def ask_bert_review(text, question):
    inputs = tokenizer.encode_plus(
        question,
        text,
        add_special_tokens=True,
        return_tensors="pt",
        max_length=512,  # Limit input length
        truncation=True  # Truncate if longer than 512
    ).to(device)

    # Forward pass through QA model
    answer_start_scores, answer_end_scores = qa_model(**inputs, return_dict=False)
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
    )
    return answer

# Load positive and negative reviews
positive_reviews = load_reviews("aclImdb/test/pos", label=1)
negative_reviews = load_reviews("aclImdb/test/neg", label=0)

# Combine and sample reviews
all_reviews = positive_reviews + negative_reviews
random.shuffle(all_reviews)
sample_reviews = random.sample(all_reviews, 5)

# Define entity-specific questions
entity_questions = {
    "acting": "What does the review say about the acting?",
    "plot": "Why is the plot praised or criticized?",
    "direction": "What does the review say about the direction?"
}

# Analyze reviews
results = []

for review, label in sample_reviews:
    # Sentiment classification
    inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    sentiment_output = sentiment_model(**inputs)
    sentiment = torch.argmax(sentiment_output.logits, dim=1).item()
    sentiment_label = "positive" if sentiment == 1 else "negative"
    actual_label = "positive" if label == 1 else "negative"

    # Entity-specific reasoning
    explanations = {}
    for aspect, question in entity_questions.items():
        explanations[aspect] = ask_bert_review(review, question)

    # Store results
    results.append({
        "review": review,
        "predicted_sentiment": sentiment_label,
        "actual_sentiment": actual_label,
        "explanations": explanations
    })

# Print results
for i, result in enumerate(results):
    print(f"Review {i + 1}:")
    print(f"Text: {result['review'][:300]}...")  # Truncate for display
    print(f"Predicted Sentiment: {result['predicted_sentiment']}")
    print(f"Actual Sentiment: {result['actual_sentiment']}")
    for aspect, explanation in result["explanations"].items():
        print(f"Reason for {aspect}: {explanation}")
    print("\n")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Review 1:
Text: Here is a movie of adventure, determination, heroism, & bravery. Plus, it's set back in the late 1800s which makes it even more interesting. It's a wonderful, adventurous storyline, and Alyssa Milano is wonderful at playing the wholesome, confident, no-nonsense Fizzy...a great role-model. This is on...
Predicted Sentiment: positive
Actual Sentiment: positive
Reason for acting: alyssa milano is wonderful
Reason for plot: excellent
Reason for direction: wonderfully


Review 2:
Text: I remember being so excited on Saturday nights when I was a kid, waiting for Dr. Who. I thought it was the best show ever made. Then, I grew up, Dr. Who went off the air, and no one I knew had ever heard of it. Then I found out there was going to be a new series. I was a little nervous about it. Was...
Predicted Sentiment: positive
Actual Sentiment: positive
Reason for acting: a little behind the rest of you
Reason for plot: i ' m a little behind the rest of you
Reason for direction: a little 